In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_dir = f"datasets/abalone"
results_dir_tanh = "results/regression/single_layer/tanh/abalone"
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
model_names_nodewise = ["Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise", "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]


full_config_path = "abalone_N3341_p8"

tanh_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh,
    include_prior=False,
)

tanh_fit_nodewise = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_nodewise,
    include_prior=False,
)

In [3]:
from utils.generate_data import load_abalone_regression_data
X, X_test, y, y_test = load_abalone_regression_data(standardized=False, frac=1.0)
# Coerce everything to plain float64 NumPy arrays
X      = np.asarray(X, dtype=float)
X_test = np.asarray(X_test, dtype=float)

# y often comes as a (n,1) DataFrame/array — flatten to (n,)
y      = np.asarray(y, dtype=float).reshape(-1)
y_test = np.asarray(y_test, dtype=float).reshape(-1)

In [ ]:
from utils.kappa_matrix import extract_model_draws, compute_shrinkage_for_W_block, shrinkage_eigs_and_df
from utils.sparsity import local_prune_weights

def compute_shrinkage_with_pruning(
    X,
    W_all, b_all, v_all,          # (D,H,p), (D,H), (D,H)
    sigma_all, tau_w_all, tau_v_all,  # (D,), (D,), (D,)
    lambda_all,                   # (D,H,p)
    activation="tanh",
    return_mats=True,             # set False if you only want summaries
    include_b1_in_Sigma: bool = True,
    include_b2_in_Sigma: bool = True,
    sparsity = 0.0
):
    """
    Loop over draws and compute R=(P+S)^{-1}P per draw using your single-draw function.
    Returns:
      R_stack : (D, N, N) with N=H*p  (if return_mats=True, else None)
      r_eigs  : (D, N)  sorted eigenvalues in [0,1]
      df_eff  : (D,)    effective dof = tr(I-R) = N - tr(R)
    """
    D, H, p = W_all.shape
    N = H * p

    R_stack = np.empty((D, N, N)) if return_mats else None
    S_stack = np.empty((D, N, N)) if return_mats else None
    P_stack = np.empty((D, N, N)) if return_mats else None
    G_stack = np.empty((D, N, N)) if return_mats else None
    shrink_stack= np.empty((D, N, N)) if return_mats else None
    r_eigs  = np.empty((D, N))
    df_eff  = np.empty(D)

    for d in range(D):
        mask = local_prune_weights(W_all[d], sparsity_level=sparsity)
        W_pruned = mask[0]*W_all[d]
        R, P, S, Sigma_y, _, _ = compute_shrinkage_for_W_block(
            X=X,
            W0=W_pruned,
            b0=b_all[d],
            v0=v_all[d],
            noise=float(sigma_all[d]),
            tau_w=float(tau_w_all[d]),
            tau_v=float(tau_v_all[d]),
            lambda_tilde=lambda_all[d],
            activation=activation,
            include_b1_in_Sigma=include_b1_in_Sigma,
            include_b2_in_Sigma=include_b2_in_Sigma,
        )
        p = np.diag(P)                       
        P_inv_sqrt = np.diag(1.0/np.sqrt(p))         
        G = P_inv_sqrt @ S @ P_inv_sqrt 
        I = np.identity(N)
        shrink_mat = np.linalg.inv(I + G)@G

        if return_mats:
            R_stack[d] = R
            S_stack[d] = S
            P_stack[d] = P
            G_stack[d] = G
            shrink_stack[d] = shrink_mat
        
        r, df = shrinkage_eigs_and_df(P, S)
        r_eigs[d] = np.sort(r)
        df_eff[d] = df

    return R_stack, S_stack, P_stack, G_stack, shrink_stack, r_eigs, df_eff


# W, b1, v, b2, noise, tau_w, tau_v, lambda_eff = extract_model_draws(
#     tanh_fit_nodewise, model='Dirichlet Horseshoe tanh nodewise'
# )
# R_DHS, S_DHS, P_DHS, G_DHS, shrink_DHS, eigs_DHS, df_eff_DHS = compute_shrinkage_with_pruning(
#     X, W, b1, v, noise, tau_w, tau_v, lambda_eff,
#     activation="tanh",
#     include_b1_in_Sigma=True,
#     include_b2_in_Sigma=True,
# )
# print("done with DHS")

# W, b1, v, b2, noise, tau_w, tau_v, lambda_eff = extract_model_draws(
#     tanh_fit_nodewise, model='Dirichlet Student T tanh nodewise'
# )
# R_DST, S_DST, P_DST, G_DST, shrink_DST, eigs_DST, df_eff_DST = compute_shrinkage_with_pruning(
#     X, W, b1, v, noise, tau_w, tau_v, lambda_eff,
#     activation="tanh",
#     include_b1_in_Sigma=True,
#     include_b2_in_Sigma=True,
# )
# print("done with DST")

W, b1, v, b2, noise, tau_w, tau_v, lambda_eff = extract_model_draws(
    tanh_fit_nodewise, model='Beta Horseshoe tanh nodewise'
)
R_BHS, S_BHS, P_BHS, G_BHS, shrink_BHS, eigs_BHS, df_eff_BHS = compute_shrinkage_with_pruning(
    X, W, b1, v, noise, tau_w, tau_v, lambda_eff,
    activation="tanh",
    include_b1_in_Sigma=True,
    include_b2_in_Sigma=True,
)
print("done with BHS")

W, b1, v, b2, noise, tau_w, tau_v, lambda_eff = extract_model_draws(
    tanh_fit_nodewise, model='Dirichlet Student T tanh nodewise'
)
R_BST, S_BST, P_BST, G_BST, shrink_BST, eigs_BST, df_eff_BST = compute_shrinkage_with_pruning(
    X, W, b1, v, noise, tau_w, tau_v, lambda_eff,
    activation="tanh",
    include_b1_in_Sigma=True,
    include_b2_in_Sigma=True,
)
print("done with BST")

In [ ]:
import os
import numpy as np

os.makedirs("Abalone_matrices", exist_ok=True)

def save_PS(model_name, P, S):
    fn = os.path.join("Abalone_matrices", f"{model_name.replace(' ', '_')}_PS.npz")
    np.savez_compressed(fn, P=np.asarray(P, dtype=np.float32), S=np.asarray(S, dtype=np.float32))
    print(f"Saved {fn}  with P,S shapes={P.shape},{S.shape}  dtype=float32")

# Call once per model (arrays are shape (4000, 160, 160))
# save_PS("Dirichlet_Horseshoe_nodewise_sparsity_90", P_DHS,   S_DHS)
# save_PS("Dirichlet_StudentT_nodewise_sparsity_90",  P_DST,   S_DST)
save_PS("Beta_Horseshoe_nodewise0", P_BHS,   S_BHS)
save_PS("Beta_StudentT_nodewise",  P_BST,   S_BST)


In [5]:
import numpy as np
from numpy.linalg import cholesky, solve
from utils.kappa_matrix import shrinkage_matrix_stable

def build_operators_from_PS(P, S):
    """
    P, S: arrays of shape (S, d, d), SPD per sample.
    Returns:
      G        : P^{-1/2} S P^{-1/2}
      shrink_PS: (P+S)^{-1} S
      shrink_G : (I+G)^{-1} G
    """
    S_, d, _ = P.shape
    G         = np.empty_like(P, dtype=np.float64)
    shrink_PS = np.empty_like(P, dtype=np.float64)
    shrink_G  = np.empty_like(P, dtype=np.float64)

    I = np.eye(d)

    for s in range(S_):
        Ps = P[s]; Ss = S[s]

        C = cholesky(Ps)            
        temp = solve(C.T, Ss)
        Gs   = solve(C, temp.T)
        G[s] = Gs

        Rs = shrinkage_matrix_stable(Ps, Ss)

        shrink_PS[s] = np.eye(Ps.shape[0]) - Rs
        
        B = I + Gs
        LB = cholesky(B)
        YB = solve(LB, Gs)
        XB = solve(LB.T, YB)
        shrink_G[s] = XB

    return G, shrink_PS, shrink_G


# Example usage after reloading a saved NPZ:
dat = np.load("Abalone_matrices/Gaussian_PS.npz")
P_gauss, S_gauss = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
G_gauss, shrink_PS_gauss, shrink_G_gauss = build_operators_from_PS(P_gauss, S_gauss)

dat = np.load("Abalone_matrices/Regularized_Horseshoe_PS.npz")
P_RHS, S_RHS = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
G_RHS, shrink_PS_RHS, shrink_G_RHS = build_operators_from_PS(P_RHS, S_RHS)

dat = np.load("Abalone_matrices/Dirichlet_Horseshoe_PS.npz")
P_DHS, S_DHS = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
G_DHS, shrink_PS_DHS, shrink_G_DHS = build_operators_from_PS(P_DHS, S_DHS)

dat = np.load("Abalone_matrices/Dirichlet_StudentT_PS.npz")
P_DST, S_DST = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
G_DST, shrink_PS_DST, shrink_G_DST = build_operators_from_PS(P_DST, S_DST)

dat = np.load("Abalone_matrices/Beta_Horseshoe_PS.npz")
P_BHS, S_BHS = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
G_BHS, shrink_PS_BHS, shrink_G_BHS = build_operators_from_PS(P_BHS, S_BHS)

dat = np.load("Abalone_matrices/Beta_StudentT_PS.npz")
P_BST, S_BST = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
G_BST, shrink_PS_BST, shrink_G_BST = build_operators_from_PS(P_BST, S_BST)

dat = np.load("Abalone_matrices/Dirichlet_Horseshoe_nodewise_PS.npz")
P_DHS_nodewise, S_DHS_nodewise = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
G_DHS_nodewise, shrink_PS_DHS_nodewise, shrink_G_DHS_nodewise = build_operators_from_PS(P_DHS_nodewise, S_DHS_nodewise)

dat = np.load("Abalone_matrices/Dirichlet_StudentT_nodewise_PS.npz")
P_DST_nodewise, S_DST_nodewise = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
G_DST_nodewise, shrink_PS_DST_nodewise, shrink_G_DST_nodewise = build_operators_from_PS(P_DST_nodewise, S_DST_nodewise)

# dat = np.load("Abalone_matrices/Beta_Horseshoe_nodewise_PS.npz")
# P_BHS_nodewise, S_BHS_nodewise = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
# G_BHS_nodewise, shrink_PS_BHS_nodewise, shrink_G_BHS_nodewise = build_operators_from_PS(P_BHS_nodewise, S_BHS_nodewise)

# dat = np.load("Abalone_matrices/Beta_StudentT_nodewise_PS.npz")
# P_BST_nodewise, S_BST_nodewise = dat["P"].astype(np.float64), dat["S"].astype(np.float64)
# G_BST_nodewise, shrink_PS_BST_nodewise, shrink_G_BST_nodewise = build_operators_from_PS(P_BST_nodewise, S_BST_nodewise)

In [6]:
# --- Traces as distributions (df_eff = tr(R) vs total shrinkage = tr(I-R)) ---
import matplotlib.pyplot as plt

# If you also want “total shrinkage”, use your SP_inv_S_* stacks (I - R):
tr_SPinvS_gauss = np.trace(shrink_PS_gauss, axis1=1, axis2=2)
tr_SPinvS_RHS   = np.trace(shrink_PS_RHS,   axis1=1, axis2=2)
tr_SPinvS_DHS   = np.trace(shrink_PS_DHS,   axis1=1, axis2=2)
tr_SPinvS_DST   = np.trace(shrink_PS_DST,   axis1=1, axis2=2)
tr_SPinvS_BHS   = np.trace(shrink_PS_BHS,   axis1=1, axis2=2)
tr_SPinvS_BST   = np.trace(shrink_PS_BST,   axis1=1, axis2=2)

tr_SPinvS_DHS_nodewise   = np.trace(shrink_PS_DHS_nodewise,   axis1=1, axis2=2)
tr_SPinvS_DST_nodewise   = np.trace(shrink_PS_DST_nodewise,   axis1=1, axis2=2)


In [ ]:
plt.figure(figsize=(8,4), dpi=150)
bins = 40
# plt.hist(tr_SPinvS_gauss, bins=bins, alpha=0.5, label="Gauss")
# plt.hist(tr_SPinvS_RHS,   bins=bins, alpha=0.5, label="RHS")
plt.hist(tr_SPinvS_DHS,   bins=bins, alpha=0.5, label="DHS", color="C2")
plt.hist(tr_SPinvS_DHS_nodewise,   bins=bins, alpha=0.5, label="DHS-node", color="C2")
plt.hist(tr_SPinvS_DST,   bins=bins, alpha=0.5, label="DST", color="C3")
plt.hist(tr_SPinvS_DST_nodewise,   bins=bins, alpha=0.5, label="DST-node", color="C3")
# plt.hist(tr_SPinvS_BHS,   bins=bins, alpha=0.5, label="BHS")
# plt.hist(tr_SPinvS_BST,   bins=bins, alpha=0.5, label="BST")
plt.xlabel(r"$tr((P+S)^{-1}S)$", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.tight_layout()
#plt.savefig("figures_for_use_in_paper/Abalone_m_eff.pdf", bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(8,4), dpi=150)
bins = 40
# plt.hist(tr_SPinvS_gauss, bins=bins, alpha=0.5, label="Gauss")
# plt.hist(tr_SPinvS_RHS,   bins=bins, alpha=0.5, label="RHS")
plt.hist(tr_SPinvS_DHS,   bins=bins, alpha=0.5, label="DHS")
plt.hist(tr_SPinvS_DST,   bins=bins, alpha=0.5, label="DST")
plt.hist(tr_SPinvS_BHS,   bins=bins, alpha=0.5, label="BHS")
plt.hist(tr_SPinvS_BST,   bins=bins, alpha=0.5, label="BST")
plt.xlabel(r"$tr((P+S)^{-1}S)$", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.tight_layout()
#plt.savefig("figures_for_use_in_paper/Abalone_m_eff.pdf", bbox_inches="tight")
plt.show()